# Hartree-Fock equations for many-electron systems

We solve the Hartree-Fock equations for a closed-shell many-electron system
\begin{equation*}
\left[\hat{T} + \hat{V}\right]\Phi = \Phi F
\end{equation*}
without resorting to the kinetic energy operator
\begin{equation*}
\hat{T} = -\frac{1}{2}\nabla^2
\end{equation*}

In the equation above $\Phi$ is a row vector of doubly occupied orbitals,
$F = \langle\Phi|\hat{T}+\hat{V}|\Phi\rangle$ is the Fock matrix, and
$\hat{V} = \hat{V}_{nuc} + 2\hat{J} - \hat{K}$ is the potential operator.
We here define the nuclear attraction potential
\begin{align*}
\hat{V}_{nuc}(r) = \sum_I \frac{-Z_I}{|R_I - r|}
\end{align*}
with $Z_I$ and $R_I$ being the nuclear charges and positions,
and the Coulomb and Exchange operators
\begin{align*}
\hat{J} \varphi_i &= \sum_j \hat{P}[\varphi_j \varphi_j^\dagger] \varphi_i \\
\hat{K} \varphi_i &= \sum_j \hat{P}[\varphi_i \varphi_j^\dagger] \varphi_j
\end{align*}
where $\hat{P}$ is the convolution operator with the Poisson kernel $P=1/r$.
We iterate the Hartree-Fock equations in their integral form
\begin{equation*}
\tilde{\Phi}^{n+1} = -2 \hat{G}^{\vec{\mu}^n}\left[\hat{V} \Phi^n + \Phi^n\left(\Lambda^n - F^n\right)\right]
\end{equation*}
where $\hat{G}^{\vec{\mu}}$ is a vector of convolution operators with the
bound-state Helmholtz kernel $G^{\mu} = e^{-\mu r}/r$ and $\mu_i^n = \sqrt{-2\lambda_i^n}$
is a positive real number. $\Lambda_{ij}^n = \lambda_i^n\delta_{ij}$ is a diagonal matrix containing
the same parameters as used in the construction of the Helmholz vector. In practice we choose
$\lambda_i^n = F_{ii}^n$ such that diagonal elements vanish in $\left(\Lambda^n - F^n\right)$.


The Fock matrix at iteration $n+1$ is computed as an update to the previous guess
\begin{align*}
\tilde{F}^{n+1} = F^n + \Delta \tilde{S}^n_1 F^n + \Delta \tilde{S}^n_2 \Lambda^n + \Delta \tilde{F}^n_{pot}
\end{align*}
where
\begin{align*}
\Delta \tilde{S}^n_1 &= \langle\Delta\Phi^n |  \Phi^n \rangle \\
\Delta \tilde{S}^n_2 &= \langle\tilde{\Phi}^{n+1} | \Delta \tilde{\Phi}^n \rangle
\end{align*}
and 
\begin{align*}
\Delta \tilde{F}^n_{pot} = \langle \tilde{\Phi}^{n+1}|\hat{V}^n | \Delta \tilde{\Phi}^n\rangle
                         + \langle \tilde{\Phi}^{n+1}|\Delta \hat{V}^n | \tilde{\Phi}^{n+1}\rangle
\end{align*}


The orbitals are orthonormalized by a Löwdin rotation using the overlap matrix
$\tilde{S} = \langle \tilde{\Phi}^{n+1}|\tilde{\Phi}^{n+1}\rangle$
\begin{align*}
\Phi^{n+1}   &= \tilde{\Phi}^{n+1} \tilde{S}^{-1/2}\\
F^{n+1}      &= (\tilde{S}^{-1/2})^\dagger \tilde{F}^{n+1} \tilde{S}^{-1/2}
\end{align*}

The energy contributions are computed as follows
\begin{align*}
E_{orb}  &= 2\ tr(F) \\
E_{en}   &= 2\ tr\left(\langle \Phi | \hat{V}_{nuc} | \Phi \rangle\right) \\
E_{coul} &= 2\ tr\left( \langle \Phi | \hat{J} | \Phi \rangle\right) \\
E_{ex}   &= - tr\left(\langle \Phi | \hat{K} | \Phi \rangle\right) \\
E_{tot}  &= E_{orb} - E_{coul} - E_{ex} \\
E_{kin}  &= E_{tot} - E_{coul} - E_{ex} - E_{en} \\
\end{align*}


Here we have implemented a simple SCF solver for kinetic-free Hartree-Fock using VAMPyR, a multiresolution analysis Python library. The implementation is fully general for many-electron (closed-shell) molecules, but the current primitive initial guess for the orbitals prohibits the use on anything more complicated than a beryllium atom.

1. Define molecule with nuclear charges and positions
2. Define the computational details for the MRA
2. Create starting guesses for the Fock matrix $F^n$ and orbitals $\Phi_n$
3. Run the SCF solver
4. Plot energy and orbital errors for each SCF cycle
5. Plot converged orbitals

In [ ]:
from vampyr import vampyr3d as vp
from SCF import scf_solver, starting_guess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Define molecule
mol = [
    {"Z": 4.0, "R": [0.0, 0.0, 0.0]}
]

# Set target precision
precision = 1.0e-3

# Define MRA basis and computational domain
k = 5                       # Polynomial order of MRA basis
world = [-20, 20]           # Computational domain [-L,L]^3 (a.u.)
MRA = vp.MultiResolutionAnalysis(order=k, box=world)

# Setup guess for the orbitals and the Fock matrix
Phi_n = starting_guess(mra=MRA, atom=mol[0], n_orbs=2, prec=precision)
F_n = np.array([
    [-1.0, 0.0],
    [ 0.0,-0.5]
]) 

In [ ]:
# Run SCF solver to optimize the orbitals
orbital_updates, energies, Phi_n = scf_solver(MRA, mol, Phi_n, F_n, precision, precision, max_iter=30)

To study the convergence we'll use pandas, an easy to use data analysis library built on numpy and matplotlib. We'll first look at how the energy converges

In [ ]:
energies_df = pd.DataFrame(energies) # Convert energies list of dictionaries to a pandas dataframe

# Calcuate differences between each row
energy_diff = energies_df.diff(axis=0)
energy_diff = energy_diff.dropna()
energy_diff = energy_diff.abs()
energy_diff = energy_diff.rename(columns={"$E_{orb}$":"$\Delta E_{orb}$","$E_{en}$":"$\Delta E_{en}$",
                                          "$E_{coul}$":"$\Delta E_{coul}$", "$E_{ex}$":"$\Delta E_{ex}$",
                                          "$E_{kin}$":"$\Delta E_{kin}$", "$E_{tot}$":"$\Delta E_{tot}$"})
energy_diff.plot(logy=True, style=".-")
plt.title("Energy convergence Beryllium", fontstyle="italic", fontsize=14,fontweight='bold')
plt.xlabel("Iteration", fontstyle="italic", fontsize=14)
plt.ylabel("Energy error", fontstyle="italic", fontsize=14)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.axhline(precision, linestyle="--", color="black")
plt.legend(ncol=2)
plt.text(1, precision*1.2, "Precision", fontsize=13)
plt.show()

We see that the total energy $E_{tot}$ converges quadratically while each of the contributions to the total energy converges linearly. 

In [ ]:
updates_df = pd.DataFrame(orbital_updates)
updates_df = updates_df.rename(columns={0:"$||\Delta \phi_0||$", 1:"$||\Delta \phi_1||$"})
updates_df.plot(logy=True, style=".-")
plt.title("Orbital convergence Beryllium", fontstyle="italic", fontsize=14,fontweight='bold')
plt.xlabel("Iteration", fontstyle="italic", fontsize=14)
plt.ylabel("Orbital error", fontstyle="italic", fontsize=14)
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.legend()
plt.axhline(precision, linestyle="--", color="black")
plt.text(0, precision*0.5, f"Precision {1.0e-6}", fontsize=13)
plt.show()

Here we see both orbitals have converged to within our convergence threshold. Next let's plot the converged orbitals. Since this is a closed shell system for Beryllium we expect to see
an s and p orbital. Let's plot it along the z-axis

In [ ]:
x = np.arange(-3.0, 3.0, 0.01)
phi_0 = [Phi_n[0]([0.0, 0.0, z_i]) for z_i in x]
phi_1 = [Phi_n[1]([0.0, 0.0, z_i]) for z_i in x]
plt.plot(x, phi_0, label="$\phi_0$")
plt.plot(x, phi_1, label="$\phi_1$")
plt.legend()
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
plt.title("Beryllium orbitals", fontstyle="italic", fontsize=12,fontweight='bold')
plt.show()